# Maximum Likelihood

## Optimization Surface

ML Estimator seeks to **best fit the data**. This is the **lowest point at the neg-probability / parameters surface**.

In [ ]:
# @title Run Simulation

N_SAMPLES = 10 # @param {type:"integer"}
N_GRID = 50 # @param {type:"integer"}
mu = 2.0 # @param {type:"number"}
std = 1.0 # @param {type:"number"}

import numpy as np
from scipy import stats
import plotly.graph_objects as go

x = np.random.normal(loc=mu,scale=std,size=(N_SAMPLES,)) # samples

fn_log_p = lambda mu,std: stats.distributions.norm.logpdf(np.expand_dims(x,-1),loc=mu,scale=std).sum(0)

mu_vals = np.linspace(0.5*mu, 1.5*mu, N_GRID)
std_vals = np.linspace(0.5*std, 1.5*std, N_GRID)
mu_vals, std_vals = np.meshgrid(mu_vals, std_vals)
logp_vals = fn_log_p(mu_vals.ravel(),std_vals.ravel()) # (N_GRID * N_GRID)
#logp_vals = logp_vals.sum(0) # (, N_GRID * N_GRID)
logp_vals = logp_vals.reshape(N_GRID,N_GRID) # (N_GRID, N_GRID)

fig = go.Figure(
    data=[go.Mesh3d(x=mu_vals.ravel(), y=std_vals.ravel(), z=-logp_vals.ravel(), color='lightpink', opacity=0.50)],
)

fig.update_layout(scene = dict(
                    xaxis_title=r'mu',
                    yaxis_title=r'sigma',
                    zaxis_title=r'logp(data)',
), title=f"negative log-probability of 10 samples from normal distribution, depending on mu and sigma."
)

fig.show()

## Numerical Optimization

Optimization algorithms **may require a good initial guess**.

In [ ]:
# @title Run Optimization

mu_init = 1.0 # @param {type:"number"}
std_init = 1.0 # @param {type:"number"}

from scipy import optimize

optim = optimize.minimize(lambda x: -fn_log_p(x[0],x[1]),x0=(mu_init,std_init))
mu_best,std_best = optim.x
print(f"best mu={mu_best:.3f},best std={std_best:.3f}")

best mu=1.509,best std=0.010


# Reparametrization

**Transforming parameters** can help for both **enforcing constraings** and **improving convergence**.

In [ ]:
# @title Run Simulation

N_SAMPLES = 10
N_GRID = 50
mu = 2.0
std = 1.0
log_std = np.log(std)


import numpy as np
from scipy import stats

x = np.random.normal(loc=mu,scale=std,size=(N_SAMPLES,)) # samples

fn_log_p = lambda mu,log_std: stats.distributions.norm.logpdf(np.expand_dims(x,-1),loc=mu,scale=np.exp(log_std)).sum(0)

mu_vals = np.linspace(0.5*mu, 1.5*mu, N_GRID)
log_std_vals = np.linspace(log_std-0.5, log_std+0.5, N_GRID)
mu_vals, log_std_vals = np.meshgrid(mu_vals, log_std_vals)
logp_vals = fn_log_p(mu_vals.ravel(),log_std_vals.ravel()) # (N_GRID * N_GRID)
#logp_vals = logp_vals.sum(0) # (, N_GRID * N_GRID)
logp_vals = logp_vals.reshape(N_GRID,N_GRID) # (N_GRID, N_GRID)

fig = go.Figure(
    data=[go.Mesh3d(x=mu_vals.ravel(), y=log_std_vals.ravel(), z=-logp_vals.ravel(), color='lightpink', opacity=0.50)],
)

fig.update_layout(scene = dict(
                    xaxis_title=r'mu',
                    yaxis_title=r'log-sigma',
                    zaxis_title=r'logp(data)',
), title=f"negative log-probability of 10 samples from normal distribution, depending on mu and log-sigma."
)

fig.show()

In [ ]:
# @title Run Optimization

mu_init = 1.0 # @param {type:"number"}
log_std_init = 100.0 # @param {type:"number"}

from scipy import optimize

optim = optimize.minimize(lambda x: -fn_log_p(x[0],x[1]),x0=(mu_init,log_std_init))
mu_best,log_std_best = optim.x
std_best = np.exp(log_std_best)
print(f"best mu={mu_best:.3f},best std={std_best:.3f}")

best mu=1.988,best std=0.723
